In [ ]:
import time
import joblib
import logging
import os
from datetime import datetime
from typing import Dict, List, Any
import pandas as pd
import wmi
from sklearn.base import BaseEstimator

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Create logs directory
os.makedirs("data/logs", exist_ok=True)

try:
    # Load the trained model
    model: BaseEstimator = joblib.load("../models/battery_check_model.pkl")
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Failed to load model: {e}")
    raise

def monitor_battery() -> None:
    """Monitor battery status and predict next check interval."""
    c = wmi.WMI()
    try:
        while True:
            for battery in c.Win32_Battery():
                # Use getattr with defaults for safety
                battery_status: Dict[str, Any] = {
                    "battery_percentage": getattr(battery, 'EstimatedChargeRemaining', 0),
                    "voltage": getattr(battery, 'DesignVoltage', 0),
                    "full_charge_capacity": getattr(battery, 'FullChargeCapacity', 0),
                }

                # Prepare features (added temperature and other missing features)
                temperature = getattr(battery, 'Temperature', 0)
                other_feature = 0  # Replace with actual feature extraction logic

                features: List = [
                    [
                        battery_status["battery_percentage"],
                        battery_status["voltage"],
                        battery_status["full_charge_capacity"],
                        temperature,
                        other_feature,
                    ]
                ]

                # Make prediction
                next_check_interval = model.predict(features)

                # Log data
                log_data: Dict = {
                    "timestamp": datetime.now(),
                    "battery_percentage": battery_status["battery_percentage"],
                    "next_check_interval": next_check_interval[0],
                }

                # Save log
                with open("data/logs/battery_data_log.txt", "a") as log_file:
                    log_file.write(
                        f"{log_data['timestamp']} | Battery: {log_data['battery_percentage']}% | "
                        f"Next Check in: {log_data['next_check_interval']} minutes\n"
                    )
                
                logger.info(f"Battery data logged: {battery_status}")

            time.sleep(300)  # Wait 5 minutes

    except Exception as e:
        logger.error(f"Error monitoring battery: {e}")
        raise

if __name__ == "__main__":
    monitor_battery()

2024-11-20 00:06:02,573 - INFO - Model loaded successfully
2024-11-20 00:06:02,840 - INFO - Battery data logged: {'battery_percentage': 55, 'voltage': '12072', 'full_charge_capacity': None}
